# Make gif of labeled/predicted boxes over point clouds

In [232]:
# Get series of frames
# For each frame
    # Run inference
    # Generate a plot
    # Save the plot to a folder
# Using PIL, make gif (also crop?)

# First try just regular with labels
# Then cropped/zoomed with labels
# Then inferences
# Then filtered

In [233]:
import numpy as np
import pandas as pd
from PIL import Image
import os

In [234]:
label_dir = ''
bin_dir = ''
image_save_dir = 'gif_images'

In [235]:
def set_axes_limited(ax, limit=50):
    # Set axes to a fixed range around the origin to limit view to +/- 'limit' meters.
    ax.set_xlim3d([-limit, limit])
    ax.set_ylim3d([-limit, limit])
    ax.set_zlim3d([-limit, limit])

In [236]:
def set_axes_equal(ax):
    """Make axes of 3D plot have equal scale so that spheres appear as spheres, cubes as cubes, etc."""
    x_limits = ax.get_xlim3d()
    y_limits = ax.get_ylim3d()
    z_limits = ax.get_zlim3d()

    x_range = abs(x_limits[1] - x_limits[0])
    x_middle = np.mean(x_limits)
    y_range = abs(y_limits[1] - y_limits[0])
    y_middle = np.mean(y_limits)
    z_range = abs(z_limits[1] - z_limits[0])
    z_middle = np.mean(z_limits)

    box_size = max([x_range, y_range, z_range]) / 2.0

    ax.set_xlim3d([x_middle - box_size, x_middle + box_size])
    ax.set_ylim3d([y_middle - box_size, y_middle + box_size])
    ax.set_zlim3d([z_middle - box_size, z_middle + box_size])

In [237]:
def convert_to_dataframe(bin_path):
    pre_filtered_data = np.fromfile(bin_path, dtype=np.float32).reshape(-1, 4) 
    columns = ['x', 'y', 'z', 'intensity']
    df = pd.DataFrame(pre_filtered_data, columns=columns)
    return df

In [238]:
def read_kitti_labels(label_path):
    bboxes = []
    with open(label_path, 'r') as file:
        for line in file:
            parts = line.strip().split()
            if parts[0] != 'DontCare':
                bbox = [float(value) for value in parts[8:15]]  # Extract the bounding box dimensions and location
                bboxes.append(bbox)
    return np.array(bboxes)

In [239]:
# Function to plot the bounding box
def plot_bbox(ax, bbox, color):
    
    x, y, z, dx, dy, dz, yaw = bbox

    
    corners = np.array([
        [x - dx/2, y - dy/2, z - dz/2],
        [x + dx/2, y - dy/2, z - dz/2],
        [x + dx/2, y + dy/2, z - dz/2],
        [x - dx/2, y + dy/2, z - dz/2],
        [x - dx/2, y - dy/2, z + dz/2],
        [x + dx/2, y - dy/2, z + dz/2],
        [x + dx/2, y + dy/2, z + dz/2],
        [x - dx/2, y + dy/2, z + dz/2]
    ])
    

    # Rotate corners around the Z-axis (around their centers)
    rotation_matrix = np.array([
        [np.cos(yaw), -np.sin(yaw), 0],
        [np.sin(yaw),  np.cos(yaw), 0],
        [0,            0,           1]
    ])

    # # Adjust the corners to be about the origin for rotation, then add the [x, y, z] back after rotation
    corners = corners - np.array([x, y, z])
    corners = np.dot(corners, rotation_matrix.T)
    corners += np.array([x, y, z])
    
    # Define the edges of the bounding box
    edges = [
        (0, 1), (1, 2), (2, 3), (3, 0),
        (4, 5), (5, 6), (6, 7), (7, 4),
        (0, 4), (1, 5), (2, 6), (3, 7)
    ]
    # Plot the edges
    for edge in edges:
        ax.plot3D(*zip(corners[edge[0]], corners[edge[1]]), color=color, linewidth=1)

In [240]:
def plot_frame(bin_path, label_path, index, offset):
    # Set up the plot
    fig = plt.figure(figsize=(30, 30))

    ax = fig.add_subplot(111, projection='3d')

    points = convert_to_dataframe(bin_path)
    ax.scatter(points['x'], points['y'], points['z'], s=0.1, c=points['intensity'], cmap='viridis', marker=',')

    for bbox in read_kitti_labels(label_path):
        plot_bbox(ax, bbox, 'b')

    # Setting axes properties
    ax.set_xlabel('X')
    ax.set_ylabel('Y')
    ax.set_zlabel('Z')
    set_axes_equal(ax)

    # Setting axes properties with a limited range
    set_axes_limited(ax)

    # Adjust the view angle if needed
    ax.view_init(elev=45, azim=210)

    

    # Save the figure
    fig.savefig(f'{image_save_dir}/{int(index) + offset:06d}.png', dpi=300)
    plt.close()

In [241]:
def get_file_indices(start_index, until_index):
    return [f'{i:06d}' for i in range(start_index, until_index)]

In [242]:
def create_and_save_plot(bin_path, label_path, index, offset=0):
    plot_frame(bin_path, label_path, index, offset)

In [243]:
def assemble_gif(image_save_dir, output_path="output.gif", duration=500, crop_box=None):
    """
    Create a GIF from sequentially numbered PNG files.
    
    Args:
        image_save_dir (str): Directory containing the numbered PNG files
        output_path (str): Where to save the resulting GIF
        duration (int): Duration for each frame in milliseconds
    """
    # Get all png files and sort them numerically
    files = [f for f in os.listdir(image_save_dir) if f.endswith('.png')]
    # This will work because files are zero-padded
    files.sort()  
    
    # Open all images
    images = []
    for filename in files:
        file_path = os.path.join(image_save_dir, filename)
        img = Image.open(file_path)
        
        # Crop if crop_box is specified
        if crop_box is not None:
            img = img.crop(crop_box)
        
        images.append(img)
    
    if images:
        # Save the GIF
        images[0].save(
            output_path,
            save_all=True,
            append_images=images[1:],
            duration=duration,
            loop=0
        )
        print(f"gif created successfully at: {output_path}")
    else:
        print("No png files found in directory")

In [244]:
def get_crop_box(image_path, crop_percentages):
    """
    Calculate pixel coordinates for cropping based on percentages.
    
    Args:
        image_path (str): Path to an image file to get dimensions from
        crop_percentages (dict): Dictionary with percentages to crop from each edge
            e.g., {'top': 10, 'bottom': 10, 'left': 20, 'right': 20}
    
    Returns:
        tuple: (left, top, right, bottom) coordinates for PIL's crop function
    """
    # Open image and get size
    with Image.open(image_path) as img:
        width, height = img.size
    
    # Calculate pixels to crop from each edge
    left_crop = int(width * crop_percentages.get('left', 0) / 100)
    right_crop = int(width * crop_percentages.get('right', 0) / 100)
    top_crop = int(height * crop_percentages.get('top', 0) / 100)
    bottom_crop = int(height * crop_percentages.get('bottom', 0) / 100)
    
    # Calculate final coordinates
    left = left_crop
    top = top_crop
    right = width - right_crop
    bottom = height - bottom_crop
    
    return (left, top, right, bottom)

In [245]:
def make_gif(start_frame, until_frame):
    # Make file indices
    indices = get_file_indices(start_frame, until_frame)
    # For each file index
    for index in indices:
        # Make bin path
        bin_path = f'../../data/eval_data/arcs/{index}.bin'
        # Make .txt path
        label_path = f'../../data/eval_data/labels/{index}.txt'
        # Make plot images
        create_and_save_plot(bin_path, label_path, index)
        
    # Then add on filtered data
    for index in indices:
        # Make bin path
        bin_path = f'../../data/eval_data/arcs_height_filtered/{index}.bin'
        # Make .txt path
        label_path = f'../../data/eval_data/labels/{index}.txt'
        # Make plot images
        create_and_save_plot(bin_path, label_path, index, offset=until_frame)
        
    first_image = os.path.join(image_save_dir, "000000.png")
    crop_percentages = {
        'top': 30,
        'bottom': 45,
        'left': 33,
        'right': 33
    }

    assemble_gif(
        image_save_dir,
        output_path="cropped_animation.gif",
        duration=150,
        crop_box=get_crop_box(first_image, crop_percentages)
    )

In [246]:
make_gif(0, 40)

gif created successfully at: cropped_animation.gif
